In [1]:
import json
import pandas as pd

# 存储模型
import pickle

# data clean
import re # punctuation,data clean

# tokenization and feature extraction
# install expacted word bag for jieba
import jieba 
jieba.load_userdict('jieba_expanded_dict.txt')
import jieba.analyse # TF-IDF 
import numpy as np # TF-IDF 
from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF

# models
from sklearn.ensemble import RandomForestClassifier # RF
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVC

# analyse
import seaborn as sns # density curve 
import matplotlib.pyplot as plt # pie chart

C:\Users\lenovo\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lenovo\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\lenovo\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 0.402 seconds.
Prefix dict has been built successfully.
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# data preprocess, run this part every time

In [2]:
balanced_df_all_cleaned_tokenization = pd.read_csv('balanced_df_all_cleaned_tokenization.csv')

In [3]:
balanced_df_all_cleaned_tokenization2 = balanced_df_all_cleaned_tokenization.copy()

In [4]:
# 取500行label为0，500行label为1
df_label_0 = balanced_df_all_cleaned_tokenization2[balanced_df_all_cleaned_tokenization2['label'] == 0].sample(n=500, random_state=41)
df_label_1 = balanced_df_all_cleaned_tokenization2[balanced_df_all_cleaned_tokenization2['label'] == 1].sample(n=500, random_state=41)

# 合并两个dataframe
balanced_df_all_cleaned_tokenization2 = pd.concat([df_label_0, df_label_1])

print(balanced_df_all_cleaned_tokenization2)

          ID  label                                      tweet_content
4358   11433      0  命 林俊杰 墙头 邓伦 墙头 越越 喜欢 点 失眠 南 印象 里 老林 厨房 白 变化 太 ...
5213   11314      0  活 牢笼 里 包括 周遭 环境 机 跳出 找点 空间 里 找然 找 东莞 空调 风扇 齐开 ...
6390   16325      0  欧美 机车 女郎 美胸 诱惑 消费 钱容力 分享 豆包 爱 表现 实挺错 感觉 养生 真 重...
9226   27982      0  发快 手 作品 華少 快手 视频 妹妹 帮 哥哥 搞定 新 嫂子 涛涛 快手 视频 朋友 生...
7988   22881      0  太累 想 做事 拒绝 事 勉强 喜欢 假装 没 听见 生讨善 山越 高路 越 陡见 风暴 中...
...      ...    ...                                                ...
12919   2595      1  段 痛苦 沉重 压抑 法 摆脱 感觉 沉重 实知 困境 夸 长久 思考 痛苦 削弱 绝 反击...
20007   9683      1  明天 医院 复查 妈妈 商量 明天 出发 事聊 两 医院 两 医生 开药 副作 妈妈 说 接...
15695   5371      1  说 清 孤独 原唯 表达 期 世界 摄影师 果吃家 饭 太 放心 苏芩 摄影师 想 温暖 利...
10397     73      1  活 越久越 发觉 性 恐怖 越越 想 活活 煎熬 新增 度 恐慌 机场 接隔点 期间 确诊 ...
15057   4733      1  生日 仪式 感然 命运 公 残酷 爱 鸭 买药 便 买 蛋糕 吃火锅 意外 收获 生日礼物 ...

[1000 rows x 3 columns]


In [5]:
# replace NaN values with an empty string
balanced_df_all_cleaned_tokenization2['tweet_content'] = balanced_df_all_cleaned_tokenization2['tweet_content'].fillna('')

In [6]:
# create a TfidfVectorizer object 将tweet_content列作为特征向量化
vectorizer = TfidfVectorizer()

In [7]:
# fit and transform the data
X = vectorizer.fit_transform(balanced_df_all_cleaned_tokenization2['tweet_content'])

In [8]:
# 将label列作为目标变量
y = balanced_df_all_cleaned_tokenization2['label']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## train SVM, only run this part first time

In [16]:
# svc = SVC(kernel='linear')

In [20]:
# 训练SVM模型
svc = SVC(kernel='linear')
svc.fit(X_train, y_train)

SVC(kernel='linear')

In [21]:
# 预测测试集的label
y_pred_svc = svc.predict(X_test)

In [22]:
# 计算SVM模型评估指标
accuracy_svc = accuracy_score(y_test, y_pred_svc)
precision_svc = precision_score(y_test, y_pred_svc)
recall_svc = recall_score(y_test, y_pred_svc)
f1_svc = f1_score(y_test, y_pred_svc)

print('\nSupport Vector Machine:')
print('Accuracy:', accuracy_svc)
print('Precision:', precision_svc)
print('Recall:', recall_svc)
print('F1 Score:', f1_svc)


Support Vector Machine:
Accuracy: 0.9166666666666666
Precision: 1.0
Recall: 0.8251748251748252
F1 Score: 0.9042145593869731


In [23]:
# normal content
# new_tweet_content = ['这是 一条 新的 正常 推文']

# depressed content
new_tweet_content = ['思诺思 舒乐安定 代开 抑郁 抑郁 疫情 期间 开药 困难 开药 断药 关注 南京 兼职 超话志道 合盆友 解释 快 超话 传送门 南京 兼职']

# feature extraction
new_X = vectorizer.transform(new_tweet_content)

# SVM
new_y_pred_svc = svc.predict(new_X)

print('\nnew_tweet_content label:')
print('SVM:', new_y_pred_svc)


new_tweet_content label:
SVM: [1]


# 存储模型

In [57]:
 # %whos # 查看变量

In [24]:
# save model
with open('svm_test.pkl', 'wb') as file:
    pickle.dump(svc, file)
# dump(): 将数据序列化到文件中

# after spliting the dataset, load model, start here

In [10]:
# loade model
with open('svm_test.pkl', 'rb') as file:
    svc = pickle.load(file)
# load(): 将序列化后的数据读出（反序列化）

In [11]:
# 使用模型
# 预测测试集的label
# y_pred_svc = loaded_model.predict(X_test)
y_pred_svc = svc.predict(X_test)

In [12]:
y_pred_svc

array([0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1], d

In [13]:
# normal content
# new_tweet_content = ['这是 一条 新的 正常 推文']

# depressed content
new_tweet_content = ['思诺思 舒乐安定 代开 抑郁 抑郁 疫情 期间 开药 困难 开药 断药 关注 南京 兼职 超话志道 合盆友 解释 快 超话 传送门 南京 兼职']

# feature extraction
new_X = vectorizer.transform(new_tweet_content)

# SVM
new_y_pred_svc = svc.predict(new_X)

print('\nnew_tweet_content label:')
print('SVM:', new_y_pred_svc)


new_tweet_content label:
SVM: [1]


In [14]:
# 计算SVM模型评估指标
accuracy_svc = accuracy_score(y_test, y_pred_svc)
precision_svc = precision_score(y_test, y_pred_svc)
recall_svc = recall_score(y_test, y_pred_svc)
f1_svc = f1_score(y_test, y_pred_svc)

print('\nSupport Vector Machine:')
print('Accuracy:', accuracy_svc)
print('Precision:', precision_svc)
print('Recall:', recall_svc)
print('F1 Score:', f1_svc)


Support Vector Machine:
Accuracy: 0.97
Precision: 1.0
Recall: 0.9470588235294117
F1 Score: 0.972809667673716


In [ ]:
# save model method2

In [68]:
s = pickle.dumps(svc)
with open('svm_test2.model','wb+') as f:#注意此处mode是'wb+'，表示二进制写入
    f.write(s)

In [11]:
s = pickle.dumps(svc)
with open('svm_test2.model','wb+') as f:#注意此处mode是'wb+'，表示二进制写入
    f.write(s)

In [ ]:
# save model method3

In [25]:
import joblib

from joblib import load

In [26]:
# 保存模型
joblib.dump(svc, 'svm_test.joblib')

['svm_test.joblib']

In [ ]:
#模型读取
loaded_model = joblib.load('svm_test.joblib')